In [32]:
import sqlalchemy
sqlalchemy.__version__

'1.3.12'

## Połączenie

In [35]:
from sqlalchemy import create_engine

In [36]:
engine = create_engine('sqlite:///test1.db', echo=True)

## deklaracja mapowania

In [37]:
from sqlalchemy.ext.declarative import declarative_base
declarative_base?

In [38]:
Base = declarative_base()

In [40]:
from sqlalchemy import Column, Integer, String


class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name, self.fullname, self.nickname)

## Tworzenie mapowania

In [41]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [42]:
Base.metadata.create_all(engine)

2020-01-08 01:30:49,010 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-01-08 01:30:49,012 INFO sqlalchemy.engine.base.Engine ()
2020-01-08 01:30:49,013 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-01-08 01:30:49,014 INFO sqlalchemy.engine.base.Engine ()
2020-01-08 01:30:49,015 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-01-08 01:30:49,018 INFO sqlalchemy.engine.base.Engine ()
2020-01-08 01:30:49,020 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-01-08 01:30:49,022 INFO sqlalchemy.engine.base.Engine ()
2020-01-08 01:30:49,024 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2020-01-08 01:30:49,026 INFO sqlalchemy.engine.base.Engine ()
2020-01-08 01:30:49,034 INFO sqlalchemy.engine.base.Engine COMMIT


## Tworzenie instancji

In [43]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')

In [44]:
ed_user.name

'ed'

In [46]:
print(ed_user.id)

None


## Tworzenie sesji

In [ ]:
from sqlalchemy.orm import sessionmaker

In [47]:
Session = sessionmaker(bind=engine)

In [48]:
session = Session()

## Dodawanie i update obiektów

In [49]:
ed_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [50]:
session.add(ed_user)

In [52]:
our_user = session.query(User).filter_by(name='ed').first()
our_user

2020-01-08 01:35:23,979 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-01-08 01:35:23,984 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-01-08 01:35:23,986 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2020-01-08 01:35:23,991 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2020-01-08 01:35:23,995 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [53]:
our_user.id

1

In [54]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [55]:
session.dirty

IdentitySet([])

In [56]:
ed_user.nickname = 'eddie'

In [58]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [57]:
session.new  

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

## Rollback

In [60]:
ed_user.name = 'Edwardo'

In [61]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')

In [62]:
session.add(fake_user)

In [63]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2020-01-08 01:39:08,005 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=?, nickname=? WHERE users.id = ?
2020-01-08 01:39:08,012 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'eddie', 1)
2020-01-08 01:39:08,014 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-01-08 01:39:08,015 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2020-01-08 01:39:08,017 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-01-08 01:39:08,019 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2020-01-08 01:39:08,021 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-01-08 01:39:08,022 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2020-01-08 01:39:08,024 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-01-08 01:39:08,025 INFO sqla

[<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fakeuser', fullname='Invalid', nickname='12345')>]

In [64]:
session.rollback()

2020-01-08 01:39:16,494 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [70]:
ed_user.name

'Edwardo'

In [72]:
ed_user in session

False

In [73]:
session.query(User).filter(User.name.in_(['Ed', 'fakeuser'])).all()

2020-01-08 01:41:22,566 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2020-01-08 01:41:22,575 INFO sqlalchemy.engine.base.Engine ('Ed', 'fakeuser')


[]

In [74]:
session.query(User).all()

2020-01-08 01:41:47,577 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2020-01-08 01:41:47,584 INFO sqlalchemy.engine.base.Engine ()


[]

In [75]:
session.add(ed_user)

In [76]:
session.query(User).all()

2020-01-08 01:42:18,615 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname, nickname) VALUES (?, ?, ?, ?)
2020-01-08 01:42:18,620 INFO sqlalchemy.engine.base.Engine (1, 'Edwardo', 'Ed Jones', 'eddie')
2020-01-08 01:42:18,626 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2020-01-08 01:42:18,629 INFO sqlalchemy.engine.base.Engine ()


[<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>]

In [77]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy')])
session.commit()

2020-01-08 01:42:48,264 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-01-08 01:42:48,282 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2020-01-08 01:42:48,284 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-01-08 01:42:48,285 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2020-01-08 01:42:48,286 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-01-08 01:42:48,287 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2020-01-08 01:42:48,289 INFO sqlalchemy.engine.base.Engine COMMIT


In [78]:
session.query(User).all()

2020-01-08 01:43:05,370 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-01-08 01:43:05,373 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2020-01-08 01:43:05,374 INFO sqlalchemy.engine.base.Engine ()


[<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='wendy', fullname='Wendy Williams', nickname='windy')>,
 <User(name='mary', fullname='Mary Contrary', nickname='mary')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>]

## Zapytania


In [79]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2020-01-08 01:43:37,435 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2020-01-08 01:43:37,444 INFO sqlalchemy.engine.base.Engine ()
Edwardo Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [80]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2020-01-08 01:43:52,136 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2020-01-08 01:43:52,143 INFO sqlalchemy.engine.base.Engine ()
Edwardo Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [81]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

2020-01-08 01:44:22,554 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2020-01-08 01:44:22,557 INFO sqlalchemy.engine.base.Engine ()
<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')> Edwardo
<User(name='wendy', fullname='Wendy Williams', nickname='windy')> wendy
<User(name='mary', fullname='Mary Contrary', nickname='mary')> mary
<User(name='fred', fullname='Fred Flintstone', nickname='freddy')> fred


In [82]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

2020-01-08 01:44:39,276 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users
2020-01-08 01:44:39,279 INFO sqlalchemy.engine.base.Engine ()
Edwardo
wendy
mary
fred


itd ..  https://docs.sqlalchemy.org/en/13/orm/tutorial.html

In [7]:
connection_data = dict(
    user="doadmin",
    password="mo8ishar2150lq59",
    host="db-kpmg-tests-do-user-2498346-0.db.ondigitalocean.com",
    port=25060,
    database="defaultdb",
    sslmode="require",
)

user="doadmin"
password="mo8ishar2150lq59"
hostname="db-kpmg-tests-do-user-2498346-0.db.ondigitalocean.com"
port=25060
database="defaultdb"
sslmode="require"

In [8]:
# !pip install sqlalchemy

In [9]:
from sqlalchemy import create_engine

db_string = f"postgresql+psycopg2://{user}:{password}@{hostname}:{port}/{database}"

db = create_engine(db_string)

# Create 
db.execute("CREATE TABLE IF NOT EXISTS films (title text, director text, year text)")  
db.execute("INSERT INTO films (title, director, year) VALUES ('Doctor Strange', 'Scott Derrickson', '2016')")

# Read
result_set = db.execute("SELECT * FROM films")  
for r in result_set:  
    print(r)

# Update
db.execute("UPDATE films SET title='Some2016Film' WHERE year='2016'")

# Delete
db.execute("DELETE FROM films WHERE year='2016'") 

('Doctor Strange', 'Scott Derrickson', '2016')


In [13]:
from sqlalchemy import create_engine  
from sqlalchemy import Column, String, Integer
from sqlalchemy.ext.declarative import declarative_base  
from sqlalchemy.orm import sessionmaker

db_string = f"postgresql+psycopg2://{user}:{password}@{hostname}:{port}/{database}"

db = create_engine(db_string)  
base = declarative_base()

class Film(base):  
    __tablename__ = 'films2'

    title = Column(String, primary_key=True)
    director = Column(String)
    year = Column(Integer)

Session = sessionmaker(db)  
session = Session()

base.metadata.create_all(db)

# Create 
doctor_strange = Film(title="Doctor Strange", director="Scott Derrickson", year=2016)  
session.add(doctor_strange)  
session.commit()


# Create 
doctor_strange = Film(title="Avengers", director="Scott Derrickson", year=2019)  
session.add(doctor_strange)  
session.commit()

# Read
films = session.query(Film)  
for film in films:  
    print(film.title)

# # Update
# doctor_strange.title = "Some2016Film"  
# session.commit()

# # Delete
# session.delete(doctor_strange)  
# session.commit()  

Doctor Strange
Avengers


In [18]:
films.filter(Film.year == 2019).all()

### Zadania.

https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_quick_guide.htm

1. W oparciu o sqlite3 utworzó tablę z filmami
2. Przy użyciu Faker utwórz 100 wpisów
3. Wyszuka

In [26]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
engine = create_engine('sqlite:///college.db', echo = True)
meta = MetaData()

students = Table(
   'students', meta, 
   Column('id', Integer, primary_key = True), 
   Column('name', String), 
   Column('lastname', String), 
)

meta.create_all(engine)


ins = students.insert().values(name = 'Ravi', lastname = 'Kapoor')
conn = engine.connect()
result = conn.execute(ins)


2020-01-08 01:21:04,987 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-01-08 01:21:04,989 INFO sqlalchemy.engine.base.Engine ()
2020-01-08 01:21:04,992 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-01-08 01:21:04,994 INFO sqlalchemy.engine.base.Engine ()
2020-01-08 01:21:04,997 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("students")
2020-01-08 01:21:04,998 INFO sqlalchemy.engine.base.Engine ()
2020-01-08 01:21:05,003 INFO sqlalchemy.engine.base.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2020-01-08 01:21:05,004 INFO sqlalchemy.engine.base.Engine ('Ravi', 'Kapoor')
2020-01-08 01:21:05,006 INFO sqlalchemy.engine.base.Engine COMMIT


In [30]:
students = session.query(students)

In [31]:
students.all()

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "students" does not exist
LINE 2: FROM students
             ^

[SQL: SELECT students.id AS students_id, students.name AS students_name, students.lastname AS students_lastname 
FROM students]
(Background on this error at: http://sqlalche.me/e/f405)